## This is a Three Bus Test System

In [ ]:
### Libraries

import pandapower as pp #import pandapower
import pandas as pd
import math
import random
import pandapower.networks as nw
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats
import warnings
import statsmodels.api as sm
from scipy.stats import truncnorm
from datetime import datetime

In [15]:
# Create an empty network
net = pp.create_empty_network(sn_mva=0.315) #create an empty network

### Buses

In [16]:
# bus1 = pp.create_bus(net, name="HV Busbar", vn_kv=110, type="b")
min_vm_pu=0.95
max_vm_pu=1.05
Feeder1={}

# GridBus = pp.create_bus(net, vn_kv=11, name="Main-Bus",min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")
FeederBus = pp.create_bus(net, vn_kv=0.4, name="Feeders-Bus",min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")

for i in range(0, 2):
    bus_id = str(i)
    vn_kv = 0.4  # Voltage level from Matpower bus data
    bus_name = f"F1-{bus_id}"
    Feeder1[i]=pp.create_bus(net, vn_kv=vn_kv, name=bus_name,min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")
    
net.bus # show bus table


,name,vn_kv,type,zone,in_service,min_vm_pu,max_vm_pu
0,Feeders-Bus,0.4,b,None,True,0.95,1.05
1,F1-0,0.4,b,None,True,0.95,1.05
2,F1-1,0.4,b,None,True,0.95,1.05


### External Grid

In [ ]:
pp.create_ext_grid(net,FeederBus , vm_pu=1.0, va_degree=0, s_sc_max_mva=10000, rx_max=0.1) # Create an external grid connection

# net.ext_grid #show external grid table

0

### Lines

In [18]:
#### LV 300c AI 31/2-C
l1_r_ohm_per_km = 0.0958   # Resistance per km
l1_x_ohm_per_km = 0.07  # Reactance per km
l1_c_nf_per_km = 0             # Charging susceptance (converted to nF/km if needed)
l1_r0_ohm_per_km = 0.34838   # Resistance per km
l1_x0_ohm_per_km = 1.2328  # Reactance per km
l1_c0_ohm_per_km = 0             # Charging susceptance (converted to nF/km if needed)
l1_max_i_ka = 0.42     # Maximum thermal current rating
l1_length_km = 0.007 

### LV 100mm AI O/H 

l2_r_ohm_per_km = 0.270922   # Resistance per km
l2_x_ohm_per_km = 0.257383  # Reactance per km
l2_c_nf_per_km = 0             # Charging susceptance (converted to nF/km if needed)
l2_r0_ohm_per_km = 0.554544   # Resistance per km
l2_x0_ohm_per_km = 1.030687  # Reactance per km
l2_c0_nf_per_km = 0             # Charging susceptance (converted to nF/km if needed)
l2_max_i_ka = 0.271     # Maximum thermal current rating
l2_length_km = 0.05
l2_length_km_2=0.035
 
### Service Line 

l3_r_ohm_per_km = 1.273   # Resistance per km
l3_x_ohm_per_km = 0.00001  # Reactance per km
l3_c_nf_per_km = 0             # Charging susceptance (converted to nF/km if needed)
l3_r0_ohm_per_km = 1.273   # Resistance per km
l3_x0_ohm_per_km = 0.00001  # Reactance per km
l3_c0_nf_per_km = 0             # Charging susceptance (converted to nF/km if needed)
l3_max_i_ka = 0.099     # Maximum thermal current rating
l3_length_km = 0.02  # Using this i will make the p.u to ohms. 


### Main Feeder Bus to Feeder Start Connection ###

pp.create_line_from_parameters(net, FeederBus, Feeder1[0], l1_length_km, l1_r_ohm_per_km, l1_x_ohm_per_km, l1_c_nf_per_km, l1_max_i_ka,
                    r0_ohm_per_km= l1_r0_ohm_per_km, x0_ohm_per_km= l1_x0_ohm_per_km, c0_nf_per_km=  l1_c0_ohm_per_km )

pp.create_line_from_parameters(net, FeederBus, Feeder1[1], l2_length_km_2, l2_r_ohm_per_km, l2_x_ohm_per_km, l2_c_nf_per_km, l2_max_i_ka,
                    r0_ohm_per_km= l2_r0_ohm_per_km, x0_ohm_per_km= l2_x0_ohm_per_km, c0_nf_per_km=  l2_c0_nf_per_km )



1

In [19]:
min_vm_pu=0.95
max_vm_pu=1.05
Feeder1_load_bus={}

for i in range(0, 2):
    for j in range (0,2):
        bus_id = str(i)
        load_id = str(j)
        vn_kv = 0.4  # Voltage level from Matpower bus data
        bus_name = f"F1-Bus-{bus_id}-Load{load_id}"
        Feeder1_load_bus[i*2+j]=pp.create_bus(net, vn_kv=vn_kv, name=bus_name,min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu, type="b")
        pp.create_line_from_parameters(net, Feeder1[i], Feeder1_load_bus[i*2+j], l3_length_km, l3_r_ohm_per_km, l3_x_ohm_per_km, l3_c_nf_per_km, l3_max_i_ka,
                    r0_ohm_per_km= l3_r0_ohm_per_km, x0_ohm_per_km= l3_x0_ohm_per_km, c0_nf_per_km=  l3_c0_nf_per_km )

# net.bus # show bus table

#### Loads


In [ ]:
for i,bus in Feeder1_load_bus.items():
    load_name = f"F1-load{bus}"
    pp.create_load(net, Feeder1_load_bus[i], p_mw=150/1000, q_mvar=150*math.tan(math.acos(0.95))/1000, name=load_name)

0 3
1 4
2 5
3 6


In [ ]:
########## Custom Create DER ##############

########## Feeder 1 ##############
sgen1 = pp.create_sgen(net, Feeder1_load_bus[0], p_mw=20/1000, q_mvar=-math.tan(math.acos(0.95))*20/1000, name="DER 1")
sgen2 = pp.create_sgen(net, Feeder1_load_bus[2], p_mw=20/1000, q_mvar=-math.tan(math.acos(0.95))*20/1000, name="DER 1")
